In [8]:
#check if cypari2 is installed or not and install it if not
import importlib.util
import sys
import os
if 'cypari2' in sys.modules:
    print(f"'cypari2' already in sys.modules")
elif (spec := importlib.util.find_spec('cypari2')) is not None:
    # If you choose to perform the actual import ...
    module = importlib.util.module_from_spec(spec)
    sys.modules['cypari2'] = module
    spec.loader.exec_module(module)
    print(f"'cypari2' has been imported")
else:
    install = input("Do you want to install cypari2 to complicate the work? (y/n): ")
    if install == 'y':
        os.system('pip install cypari2')
    else:
        print('[!]sorry we can complit work')
        sys.exit()

'cypari2' already in sys.modules


In [9]:
from cypari2 import Pari
pari = Pari()

In [10]:
#select the ECDSA element
class ECDSASys:
    #here we choose only a and b because in ECDSA the GCD(p, order) = 1
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.p = pari.randomprime(2**160)
        self.EC = pari.ellinit([self.a, self.b], self.p)
        self.q = pari.ellcard(self.EC)
        while pari.isprime(self.q) != 1:
            self.p = pari.randomprime(2**160)
            self.EC = pari.ellinit([self.a, self.b], self.p)
            self.q = pari.ellcard(self.EC)
        self.G = str(pari.lift(pari.ellgenerators(self.EC)))[1:-1]
    def system(self):
        return {'a': self.a,
                'b': self.b,
                'p': self.p,
                'order': self.q,
                'G': self.G
        }
    
            

In [11]:
ecdsa = ECDSASys(0, 13)
par = ecdsa.system()
print(par)

{'a': 0, 'b': 13, 'p': 1069707925093910102740370644892186637340254553591, 'order': 1069707925093910102740371918449991171611302228159, 'G': '[1053766401548643417009905698147036239732558331719, 343094502971481818875477243358095088932820259642]'}


In [12]:
class ECDSAUser:
    def __init__(self, par):
        self.par = par
        self._d = pari.random(self.par['order'])
        self.EC = pari.ellinit([self.par['a'], self.par['b']], self.par['p'])
        self.A = pari.lift(pari.ellmul(self.EC, self.par['G'], self._d))
    def pubKey(self):
        return { 'a': self.par['a'],
                'b': self.par['b'],
                'p': self.par['p'],
                'order': self.par['order'],
                'G': self.par['G'],
                'A': self.A,
        }
    #the signing of an information
    def sign(self, h):
        k = pari.random(self.par['order'])
        #compute R = G.K
        R = str(pari.lift(pari.ellmul(self.EC, self.par['G'], k)))
        Xr = int(R[1: R.find(',')])
        s = ((h + self._d*Xr)*pari.lift(pow(k, self.par['order']-2, self.par['order'])))%self.par['order']
        return [Xr, s]
    #make the verification method static because it is independent of the global class
    @staticmethod
    #verify a signature
    def verification(KeyPub, sign, h):
        Ec = pari.ellinit([KeyPub['a'], KeyPub['b']], KeyPub['p'])
        w = pari.lift(pow(sign[1], KeyPub['order'] - 2, KeyPub['order']))
        u1 = (w*h)%KeyPub['order']
        u2 = (sign[0]*w)%KeyPub['order']
        P = str(pari.lift(pari.lift(pari.elladd(Ec, pari.ellmul(Ec, KeyPub['G'], u1), pari.ellmul(Ec, KeyPub['A'], u2)))))
        if P[1:P.find(',')] == str(sign[0]):
            return 'Signature Valid'
        else:
            return 'Signature Invalid'

In [13]:
ecdsaUser1 = ECDSAUser(par)
user1Key = ecdsaUser1.pubKey()
print(user1Key)
sign1 = ecdsaUser1.sign(12)
print(f'signatur is: {sign1}')
print("_"*80)
ecdsaUser2 = ECDSAUser(par)
user2Key = ecdsaUser2.pubKey()
print(user2Key)
sign2 = ecdsaUser2.sign(12)
print(f'signatur is: {sign2}')
print('*'*80)
print(ecdsaUser1.verification(user2Key, sign2, 12))
print(ecdsaUser2.verification(user1Key, sign1, 12))

{'a': 0, 'b': 13, 'p': 1069707925093910102740370644892186637340254553591, 'order': 1069707925093910102740371918449991171611302228159, 'G': '[1053766401548643417009905698147036239732558331719, 343094502971481818875477243358095088932820259642]', 'A': [288343987260938234180578995984449310532471062429, 278880706501126084564057258095075233135145327389]}
signatur is: [870604441365925023146509316504073023355869463242, 429180713947345168201779069257159463867052806843]
________________________________________________________________________________
{'a': 0, 'b': 13, 'p': 1069707925093910102740370644892186637340254553591, 'order': 1069707925093910102740371918449991171611302228159, 'G': '[1053766401548643417009905698147036239732558331719, 343094502971481818875477243358095088932820259642]', 'A': [286725149514983995021790124522530181764371753352, 15753818846136929686153724157089056290305137104]}
signatur is: [980707111646914543298457483529032176417149139051, 2260207690454195896522942641632930610830